In [ ]:
# !pip install -U transformers

In [1]:
import pandas as pd
from typing import List, Dict, Tuple, Optional
import json
import re
import torch
import time

In [2]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/stanfordnlp/coqa/" + splits["train"])

In [3]:
rows_numb = []
for row in range(len(df['questions'])):
    if len(df['questions'][row]) == 20:
        rows_numb.append(row)

In [ ]:
rows_100 = rows_numb[:100]

In [4]:
rows_1000 = rows_numb[:1000]

In [ ]:
# qq = [
# "When was the Vat formally opened?",
# "what is the library for?",
# "for what subjects?",
# "and?",
# "what was started in 2014?",
# "how do scholars divide the library?",
# "how many?",
# "what is the official name of the Vat?",
# "where is it?",
# "how many printed books does it contain?",
# "when were the Secret Archives moved from the rest of the library?",
# "how many items are in this secret collection?",
# "Can anyone use this library?",
# "what must be requested to view?",
# "what must be requested in person or by mail?",
# "of what books?",
# "What is the Vat the library of?",
# "How many books survived the Pre Lateran period?",
# "what is the point of the project started in 2014?",
# "what will this allow?"
# ]

# my_questions = qq[:int(len(qq) * 0.6)]
# my_questions

In [5]:
def extract_json_from_response(response: str) -> Optional[Dict]:
    """Extract and parse JSON from LLM response with error handling."""
    try:
        # Try direct JSON parsing first
        return json.loads(response)
    except json.JSONDecodeError:
        # Look for JSON-like structure in the response
        json_match = re.search(r'\{[^{}]*\}', response)
        if json_match:
            try:
                return json.loads(json_match.group(0))
            except json.JSONDecodeError:
                pass
    
    # Fallback: try to find confidence value directly
    confidence_match = re.search(r'"confidence":\s*([0-9]*\.?[0-9]+)', response)
    if confidence_match:
        return {"confidence": float(confidence_match.group(1))}
    
    return None

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="Qwen/Qwen3-4B-Instruct-2507")

2026-01-18 04:43:20.169333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768711400.398076      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768711400.464153      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768711401.025249      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768711401.025289      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768711401.025291      55 computation_placer.cc:177] computation placer alr

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# from transformers import pipeline

# pipe = pipeline("text-generation", model="Qwen/Qwen2.5-7B-Instruct")

In [ ]:
# def extract_main_topic(pipe, questions: list[str], temperature: float = 0.7) -> str:
#     """Extract the main topic/theme from a set of related questions from the same user context."""
    
#     system_prompt = """
# You are a topic extraction expert. Your task is to analyze multiple questions from the same user and identify the single main topic or theme that connects all of them.

# **Instructions:**
# 1. Analyze all the provided questions together
# 2. Identify the core subject, domain, or theme that unifies these questions
# 3. The questions are from the same user context, so they likely revolve around one central topic
# 4. Output ONLY a JSON object with a single "topic" field
# 5. The main topic should be concise (2-8 words maximum) and capture the essence of all questions

# **Output format (STRICTLY follow this - no other text):**
# {
#   "topic": "possibility of intelligent life existing on other planets"
# }
# """
    
#     questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(questions)])
    
#     user_prompt = f"""
# **Questions from the same user context:**
# {questions_formatted}

# Now analyze these questions and extract the single main topic that connects them all:
# """
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]

#     response = pipe(
#             messages,
#             temperature=temperature,
#             do_sample=False,
#             pad_token_id=pipe.tokenizer.eos_token_id
#         )
        
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     result = json.loads(generated_text)
#     if "topic" in result:
#         return str(result["topic"]).strip()
        
#     # topic_match = re.search(r'"topic":\s*"([^"]+)"', generated_text)
#     #     if topic_match:
#     #         return topic_match.group(1).strip()

In [ ]:
# topic = extract_main_topic(pipe, my_questions)
# topic

In [ ]:
# def extract_topic_aspects(pipe, topic: str, questions: list[str], temperature: float = 0.7) -> list[str]:
#     """Extract specific aspects/details of a known topic that the user is interested in based on their questions."""
    
#     system_prompt = """
# You are an aspect extraction expert. Your task is to analyze questions about a specific topic and identify the key aspects, details, or dimensions of that topic the user is interested in.

# **Instructions:**
# 1. You are given a main topic and multiple questions about that topic from the same user
# 2. Analyze the questions to identify what specific aspects, details, or sub-topics of the main topic the user cares about
# 3. Focus on concrete details mentioned or implied in the questions
# 4. Output ONLY a JSON object with a single "aspects" field containing a list of strings
# 5. Each aspect should be concise (2-6 words) and represent a distinct detail or dimension of the main topic
# 6. Include only aspects that are clearly supported by the questions

# **Output format (STRICTLY follow this - no other text):**
# {
#   "aspects": ["scientific reasoning for existence", "detection methods and technology", "expert predictions and timeline"]
# }

# **Examples:**
# - Topic: "possibility of intelligent life existing on other planets" + Questions about evidence, which planet, methods, timeline, and experts → [age of universe, Earth-like planets,  radio telescopes, timeline for contact]
# - Topic: "university" + Questions about founders, buildings, programs → ["founding history", "campus architecture", "academic programs"]
# """
    
#     questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(questions)])
    
#     user_prompt = f"""
# **Main Topic:** "{topic}"

# **User Questions about this topic:**
# {questions_formatted}

# Now analyze these questions and extract the specific aspects or details of "{topic}" that the user is interested in:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     response = pipe(
#         messages,
#         temperature=temperature,
#         do_sample=False,
#         pad_token_id=pipe.tokenizer.eos_token_id
#     )
    
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     # Extract JSON from response
#     result = json.loads(generated_text)
#     if "aspects" in result and isinstance(result["aspects"], list):
#         return [str(aspect).strip() for aspect in result["aspects"] if aspect]

In [ ]:
# aspects = extract_topic_aspects(pipe, topic, my_questions)
# # aspects

In [ ]:
# def generate_questions(pipe, topic: str, aspects: list[str], questions: list[str], temperature: float = 0.7) -> list[str]:
#     """Generate possible follow-up questions a user might ask based on topic, aspects, and previously asked questions."""
    
#     system_prompt = """
# You are a question generation expert. Your task is to generate new, relevant questions that a user might ask about a specific topic, based on the aspects they're interested in and avoiding questions they've already asked.

# **Instructions:**
# 1. You are given a main topic, key aspects of that topic, and questions the user has already asked
# 2. Generate 8 NEW questions that:
#    - Are relevant to the main topic
#    - Cover the specified aspects that haven't been fully explored yet
#    - Are different from the already asked questions (avoid rephrasing the same questions)
#    - Sound natural and like something a real user would ask
#    - Focus on details, specifics, or related information the user might want to know next
# 3. Output ONLY a JSON object with a single "questions" field containing a list of strings
# 4. Each question should be clear, concise, and self-contained

# **Output format (STRICTLY follow this - no other text):**
# {
#   "questions": [
#     "What galaxy do we live in?",
#     "How many stars does it have?",
#     "Does Shostak believe beings from space could make contact with Earth soon?"
#   ]
# }

# **Important Guidelines:**
# - DO NOT repeat or rephrase the already asked questions
# - DO cover different aspects or dive deeper into aspects that weren't fully explored
# - DO make questions specific and actionable
# - DO maintain a natural, conversational tone
# - DO focus on what the user might want to know next based on their interests
# """
    
#     aspects_formatted = ", ".join([f'"{aspect}"' for aspect in aspects])
#     questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(questions)])
    
#     user_prompt = f"""
# **Main Topic:** "{topic}"

# **Key Aspects of Interest:** {aspects_formatted}

# **Questions Already Asked:**
# {questions_formatted}

# Now generate new, relevant questions that the user might ask next. These questions should explore the topic and aspects in new ways, avoiding repetition of the already asked questions:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     # try:
#     #     response = pipe(
#     #         messages,
#     #         temperature=temperature,
#     #         do_sample=True,
#     #         pad_token_id=pipe.tokenizer.eos_token_id
#     #     )
        
#     #     # Extract generated text
#     #     if isinstance(response, list) and len(response) > 0:
#     #         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#     #             generated_text = response[0]['generated_text'][-1]['content']
#     #         else:
#     #             generated_text = str(response[0])
#     #     else:
#     #         generated_text = str(response)
        
#     #     # Extract JSON from response
#     #     result = json.loads(generated_text)
#     #     if "questions" in result and isinstance(result["questions"], list):
#     #         print('sfsegergerg1111')
#     #         return [str(q).strip() for q in result["questions"] if q and q.strip()]
#     # except json.JSONDecodeError:
#     #     # Try to find JSON pattern in the response
#     #     json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#     #     if json_match:
#     #         result = json.loads(json_match.group(0))
#     #         if "questions" in result and isinstance(result["questions"], list):
#     #             print('sfsegergerg2222')
#     #             return [str(q).strip() for q in result["questions"] if q and q.strip()]

#     response = pipe(
#         messages,
#         temperature=temperature,
#         do_sample=True,
#         pad_token_id=pipe.tokenizer.eos_token_id
#     )
    
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     # Extract JSON from response
#     try:
#         result = json.loads(generated_text)
#         if "questions" in result and isinstance(result["questions"], list):
#             # Filter out empty questions and limit to 20
#             questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#             return questions
#     except json.JSONDecodeError:
#         # Try to find JSON pattern in the response
#         json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#         if json_match:
#             result = json.loads(json_match.group(0))
#             if "questions" in result and isinstance(result["questions"], list):
#                 questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#                 return questions
    
#     # # Fallback: try to extract questions directly from text
#     # questions_match = re.search(r'"questions":\s*\[([^\]]+)\]', generated_text)
#     # if questions_match:
#     #     questions_str = questions_match.group(1)
#     #     # Extract individual questions from the list string
#     #     questions = [q.strip().strip('"').strip("'") for q in questions_str.split(',') if q.strip()]
#     #     return questions

In [ ]:
# gen_questions = generate_questions(pipe, topic, aspects, my_questions)
# gen_questions

In [ ]:
# gen_questions = generate_questions(pipe, topic, aspects, my_questions)
# gen_questions

In [ ]:
# def generate_questions2(pipe, topic: str, aspects: list[str], temperature: float = 0.7, max_new_tokens: int = 200) -> list[str]:
#     """Generate possible follow-up questions a user might ask based on topic, aspects, and previously asked questions."""
    
#     system_prompt = """
# You are a question generation expert. Your task is to generate new, relevant questions that a user might ask about a specific topic, based on the aspects they're interested in.

# **Instructions:**
# 1. You are given a main topic and key aspects of that topic
# 2. Generate 5-8 NEW questions that:
#    - Are relevant to the main topic
#    - Cover the specified aspects that haven't been fully explored yet
#    - Sound natural and like something a real user would ask
#    - Focus on details, specifics, or related information the user might want to know next
# 3. Output ONLY a JSON object with a single "questions" field containing a list of strings
# 4. Each question should be clear, concise, and self-contained

# **Output format (STRICTLY follow this - no other text):**
# {
#   "questions": [
#     "What galaxy do we live in?",
#     "How many stars does it have?",
#     "Does Shostak believe beings from space could make contact with Earth soon?"
#   ]
# }

# **Important Guidelines:**
# - DO cover different aspects or dive deeper into aspects that weren't fully explored
# - DO make questions specific and actionable
# - DO maintain a natural, conversational tone
# - DO focus on what the user might want to know next based on their interests
# """
    
#     aspects_formatted = ", ".join([f'"{aspect}"' for aspect in aspects])
#     # questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(questions)])
    
#     user_prompt = f"""
# **Main Topic:** "{topic}"

# **Key Aspects of Interest:** {aspects_formatted}

# Now generate new, relevant questions that the user might ask next. These questions should explore the topic and aspects in new ways:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     try:
#         response = pipe(
#             messages,
#             temperature=temperature,
#             max_new_tokens=max_new_tokens,
#             do_sample=True,
#             pad_token_id=pipe.tokenizer.eos_token_id
#         )
        
#         # Extract generated text
#         if isinstance(response, list) and len(response) > 0:
#             if isinstance(response[0], dict) and 'generated_text' in response[0]:
#                 generated_text = response[0]['generated_text'][-1]['content']
#             else:
#                 generated_text = str(response[0])
#         else:
#             generated_text = str(response)
        
#         # Extract JSON from response
#         result = json.loads(generated_text)
#         if "questions" in result and isinstance(result["questions"], list):
#             return [str(q).strip() for q in result["questions"] if q and q.strip()]
#     except json.JSONDecodeError:
#         # Try to find JSON pattern in the response
#         json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#         if json_match:
#             result = json.loads(json_match.group(0))
#             if "questions" in result and isinstance(result["questions"], list):
#                 return [str(q).strip() for q in result["questions"] if q and q.strip()]

In [ ]:
# gen_questions3 = generate_questions2(pipe, topic, aspects)
# gen_questions3

In [ ]:
# confidence = compare_questions(pipe, 'Has the size or composition of the secret collection changed over the years?', 'How many books survived the Pre Lateran period?')
# confidence

In [7]:
def compare_questions(pipe, question1: str, question2: str, 
                     temperature: float = 0.7, max_new_tokens: int = 100) -> float:

    sys_prompt = """
You are a semantic equivalence evaluator. Your task is to compare two questions and determine the confidence that they are semantically equivalent.

**Instructions:**
1. Compare the meaning and intent of the two questions
2. Determine if they're asking for the same information and would yield the same answer
3. Output ONLY a JSON object with a single "confidence" field
4. The confidence must be a number between 0 and 1

**Explanation of confidence scores:**
- 0.9-1.0: Definitely equivalent (same meaning, intent, and expected answer)
- 0.7-0.9: Highly likely equivalent (minor wording differences only)
- 0.5-0.7: Possibly equivalent (similar intent but different phrasing)
- 0.3-0.5: Unlikely equivalent (related but different aspects)
- 0.0-0.3: Definitely different (completely different questions)

**Output format (STRICTLY follow this - no other text):**
{
  "confidence": 0.85
}
"""

    prompt = f"""
**Questions to compare:**
Question 1: "{question1}"
Question 2: "{question2}"
"""

    messages = [
    {"role": "system", "content": sys_prompt.strip()},
    {"role": "user", "content": prompt.strip()}
]

    response = pipe(
        messages,
        temperature=temperature,
        do_sample=False
    )

    # Extract generated text
    if isinstance(response, list) and len(response) > 0:
        if isinstance(response[0], dict) and 'generated_text' in response[0]:
            # Handle pipeline output format
            generated_text = response[0]['generated_text'][-1]['content']
        else:
            # Handle different output format
            generated_text = str(response[0])
    else:
        generated_text = str(response)
    
    # Extract JSON from response
    result = extract_json_from_response(generated_text)
    
    if result and "confidence" in result:
        confidence = float(result["confidence"])
        # Clamp to 0-1 range
        return max(0.0, min(1.0, confidence))
    else:
        print(f"Warning: Could not parse confidence from response: {generated_text}")
        return 0.0

In [ ]:
# a = compare_questions(pipe, 'What is the Vat the library of?', 'What is the Vat the library of?')

In [ ]:
# df

In [ ]:
# results = {}

In [ ]:
# for row in rows_100:
#     print(row)
#     questions = df['questions'].loc[row].tolist()[:int(len(df['questions'].loc[row].tolist()) * 0.6)]
#     topic = extract_main_topic(pipe, questions)
#     aspects = extract_topic_aspects(pipe, topic, questions)
#     gen_questions = generate_questions(pipe, topic, aspects, questions)

#     comparisons = {}
#     remaining_questions = df['questions'].loc[row].tolist()[int(len(df['questions'].loc[row].tolist()) * 0.6):]
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     results[f'row {row}'] = {
#     "topic": topic,
#     "aspects": aspects,
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
# results_df = pd.DataFrame(results)
# results_df.T

In [ ]:
# results_df.T.to_csv('/kaggle/working/results_app1.csv')

In [ ]:
# def generate_questions_from_text(pipe, text: str, temperature: float = 0.7, max_new_tokens: int = 500) -> list[str]:
#     """Generate 10-15 relevant questions based on provided text content."""
    
#     system_prompt = """
# You are a question generation expert. Your task is to analyze a given text passage and generate relevant, insightful questions that a reader might ask about the content.

# **Instructions:**
# 1. Carefully read and understand the provided text
# 2. Generate 8 diverse questions that:
#    - Cover key facts, details, and concepts from the text
#    - Explore implications, context, and deeper meaning
#    - Ask about specific examples, dates, names, or events mentioned
#    - Probe relationships between different elements in the text
#    - Question assumptions or explore alternative perspectives
#    - Are natural and conversational (not robotic)
# 3. Ensure questions are:
#    - Clear, concise, and self-contained
#    - Based directly on the text content (don't invent facts)
#    - Varied in type (who, what, when, where, why, how, etc.)
#    - Appropriate for the text's complexity and tone
# 4. Output ONLY a JSON object with a single "questions" field containing a list of strings
# 5. Include exactly 8 questions - no more, no less

# **Output format (STRICTLY follow this - no other text):**
# {
#   "questions": [
#     "What galaxy do we live in?",
#     "How many stars does it have?",
#     "Does Shostak believe beings from space could make contact with Earth soon?"
#   ]
# }
# """
    
#     user_prompt = f"""
# **Text to analyze:**
# {text}

# Now generate 10-15 thoughtful questions based on this text content:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     try:
#         response = pipe(
#             messages,
#             temperature=temperature,
#             max_new_tokens=max_new_tokens,
#             do_sample=True,
#             pad_token_id=pipe.tokenizer.eos_token_id
#         )
        
#         # Extract generated text
#         if isinstance(response, list) and len(response) > 0:
#             if isinstance(response[0], dict) and 'generated_text' in response[0]:
#                 generated_text = response[0]['generated_text'][-1]['content']
#             else:
#                 generated_text = str(response[0])
#         else:
#             generated_text = str(response)
        
#         # Extract JSON from response
#         try:
#             result = json.loads(generated_text)
#             if "questions" in result and isinstance(result["questions"], list):
#                 # Filter out empty questions and limit to 20
#                 questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#                 return questions
#         except json.JSONDecodeError:
#             # Try to find JSON pattern in the response
#             json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#             if json_match:
#                 try:
#                     result = json.loads(json_match.group(0))
#                     if "questions" in result and isinstance(result["questions"], list):
#                         questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#                         return questions
#                 except json.JSONDecodeError:
#                     pass
        
#         # Fallback: try to extract questions directly from text
#         questions_match = re.search(r'"questions":\s*\[([^\]]+)\]', generated_text)
#         if questions_match:
#             questions_str = questions_match.group(1)
#             # Extract individual questions from the list string
#             questions = [q.strip().strip('"').strip("'") for q in questions_str.split(',') if q.strip()]
#             return questions
        
#         print(f"Warning: Could not parse generated questions from response: {generated_text}")
#         return []
            
#     except Exception as e:
#         print(f"Error generating questions from text: {e}")
#         return []

In [ ]:
# def generate_questions_from_text_with_questions(pipe, text: str, questions: list[str], temperature: float = 0.7, max_new_tokens: int = 500) -> list[str]:
#     """Generate 10-15 relevant questions based on provided text content."""
    
#     system_prompt = """
# You are a question generation expert. Your task is to analyze a given text passage and generate relevant, insightful questions that a reader might ask about the content, while avoiding questions that have already been asked.

# **Instructions:**
# 1. Carefully read and understand the provided text
# 2. Review the list of already asked questions to avoid duplication
# 3. Generate 8 diverse questions that:
#    - Cover key facts, details, and concepts from the text that haven't been explored yet
#    - Explore implications, context, and deeper meaning not covered in existing questions
#    - Ask about specific examples, dates, names, or events mentioned in the text
#    - Probe relationships between different elements in the text
#    - Question assumptions or explore alternative perspectives
#    - Are natural and conversational (not robotic)
# 4. Ensure questions are:
#    - Clear, concise, and self-contained
#    - Based directly on the text content (don't invent facts)
#    - Varied in type (who, what, when, where, why, how, etc.)
#    - Appropriate for the text's complexity and tone
#    - COMPLETELY DIFFERENT from the already asked questions (no rephrasing or similar intent)
# 5. Output ONLY a JSON object with a single "questions" field containing a list of strings
# 6. Include exactly 8 questions - no more, no less

# **Output format (STRICTLY follow this - no other text):**
# {
#   "questions": [
#     "What galaxy do we live in?",
#     "How many stars does it have?",
#     "Does Shostak believe beings from space could make contact with Earth soon?"
#   ]
# }
# """

#     questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(questions)])
    
#     user_prompt = f"""
# **Text to analyze:**
# {text}

# **Questions Already Asked:**
# {questions_formatted}

# Now generate 8 thoughtful questions based on this text content:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     response = pipe(
#         messages,
#         temperature=temperature,
#         max_new_tokens=max_new_tokens,
#         do_sample=True,
#         pad_token_id=pipe.tokenizer.eos_token_id
#     )
    
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     # Extract JSON from response
#     try:
#         result = json.loads(generated_text)
#         if "questions" in result and isinstance(result["questions"], list):
#             # Filter out empty questions and limit to 20
#             questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#             return questions
#     except json.JSONDecodeError:
#         # Try to find JSON pattern in the response
#         json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#         if json_match:
#             result = json.loads(json_match.group(0))
#             if "questions" in result and isinstance(result["questions"], list):
#                 questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
#                 return questions

    
#     # # Fallback: try to extract questions directly from text
#     # questions_match = re.search(r'"questions":\s*\[([^\]]+)\]', generated_text)
#     # if questions_match:
#     #     questions_str = questions_match.group(1)
#     #     # Extract individual questions from the list string
#     #     questions = [q.strip().strip('"').strip("'") for q in questions_str.split(',') if q.strip()]
#     #     return questions[:20]
    
#     # print(f"Warning: Could not parse generated questions from response: {generated_text}")
#     # return []

In [ ]:
# gen_q = generate_questions_from_text(pipe, df['story'][0])

In [ ]:
# gen_q

In [ ]:
# reults_app3 = {}

In [ ]:
# for row in range(100):
#     print(row)
#     gen_questions = generate_questions_from_text(pipe, df['story'][row])

#     comparisons = {}
#     remaining_questions = df['questions'].loc[row].tolist()[int(len(df['questions'].loc[row].tolist()) * 0.6):]
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     reults_app3[f'row {row}'] = {
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
# results_app3_df_last = pd.DataFrame(reults_app3)

In [ ]:
# results_app3_df_last.to_csv('/kaggle/working/results_app3_df_last.csv')

In [ ]:
# results_app3 = {}

In [ ]:
# for row in rows_100:
#     print(row)
#     remaining_questions = df['questions'].loc[row].tolist()[int(len(df['questions'].loc[row].tolist()) * 0.6):]
#     gen_questions = generate_questions_from_text_with_questions(pipe, df['story'][row], remaining_questions)

#     comparisons = {}
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     results_app3[f'row {row}'] = {
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
# results_app3_df_new = pd.DataFrame(results_app3)

In [ ]:
# results_app3_df_new.to_csv('/kaggle/working/results_app3_df_new.csv')

In [ ]:
# for row in range(20,100):
#     print(row)
#     questions = df['questions'].loc[row].tolist()[:int(len(df['questions'].loc[row].tolist()) * 0.6)]
#     topic = extract_main_topic(pipe, questions)
#     aspects = extract_topic_aspects(pipe, topic, questions)
#     gen_questions = generate_questions(pipe, topic, aspects, questions)

#     comparisons = {}
#     remaining_questions = df['questions'].loc[row].tolist()[int(len(df['questions'].loc[row].tolist()) * 0.6):]
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     results[f'row {row}'] = {
#     "topic": topic,
#     "aspects": aspects,
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
# def extract_text_main_topic(pipe, text: str, temperature: float = 0.7) -> str:
#     """Extract the single main topic/theme from a provided text passage."""
    
#     system_prompt = """
# You are a topic extraction expert. Your task is to analyze a text passage and identify the single main topic or central theme that best represents the entire content.

# **Instructions:**
# 1. Carefully read and understand the entire text passage
# 2. Identify the core subject, central theme, or primary focus of the text
# 3. The main topic should be the overarching concept that ties all parts of the text together
# 4. Output ONLY a JSON object with a single "main_topic" field
# 5. The main topic should be concise (2-5 words maximum) and capture the essence of the text
# 6. Be specific and accurate - avoid generic terms like "story" or "text"

# **Output format (STRICTLY follow this - no other text):**
# {
#   "main_topic": "possibility of intelligent life existing on other planets"
# }
# """
    
#     user_prompt = f"""
# **Text passage to analyze:**
# {text}

# Now identify the single main topic that best represents this entire text:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]

#     response = pipe(
#             messages,
#             temperature=temperature,
#             do_sample=False,
#             pad_token_id=pipe.tokenizer.eos_token_id
#         )
        
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     result = json.loads(generated_text)
#     if "main_topic" in result:
#         return str(result["main_topic"]).strip()

In [ ]:
# main_topic = extract_text_main_topic(pipe, df['story'][0])
# main_topic

In [ ]:
# reults_main_topics = {}

In [ ]:
# for row in rows_1000:
#     if row % 10 == 0:
#         print(row)
#     main_topic = extract_text_main_topic(pipe, df['story'][row])

#     reults_main_topics[f'row {row}'] = {
#     "main_topic": main_topic
#     }

In [ ]:
# reults_main_topics

In [ ]:
# reults_main_topics_df = pd.DataFrame(reults_main_topics)

In [ ]:
# reults_main_topics_df.T.to_csv('/kaggle/working/reults_main_topics_df.csv')

In [ ]:
# reults_main_topics_df = pd.read_csv('/kaggle/input/reults-main-topics-df/reults_main_topics_df.csv')
# all_topics = reults_main_topics_df['main_topic'].tolist()

In [8]:
results_main_topics_df = pd.read_csv('/kaggle/input/1000-topics-20q/reults_main_topics_df.csv')
all_topics = results_main_topics_df['main_topic'].tolist()
row_identifiers = results_main_topics_df['Unnamed: 0'].str.extract(r'row\s*(\d+)')[0].astype(int).tolist()

In [9]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN
import numpy as np
from scipy.spatial.distance import cosine

def cluster_topics_by_embeddings(topics_list, row_identifiers, min_cluster_size=3, eps=0.4):
    """
    Cluster topics using sentence embeddings and DBSCAN
    Returns: dict mapping cluster_id to list of (dialogue_index, topic)
    """
    # Load pre-trained model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Generate embeddings
    embeddings = model.encode(topics_list, show_progress_bar=True)
    
    # Cluster using DBSCAN (good for varying cluster sizes)
    clustering = DBSCAN(eps=eps, min_samples=min_cluster_size, metric='cosine')
    cluster_labels = clustering.fit_predict(embeddings)
    
    # Organize results
    clusters = {}
    for idx, (label, topic) in enumerate(zip(cluster_labels, topics_list)):
        row_id = row_identifiers[idx] 
        if label == -1:  # Noise points
            continue
        if label not in clusters:
            clusters[label] = []
        clusters[label].append((row_id, topic))
    
    return clusters

In [10]:
clusters = cluster_topics_by_embeddings(all_topics, row_identifiers)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [43]:
# clusters

{np.int64(0): [(95, 'family life and recent events'),
  (321, 'reconciliation of justice and fate'),
  (407, 'family interaction and responsibility'),
  (489, "brothers' relationship and dynamics"),
  (587, 'family secrets and silent tensions'),
  (656, 'Family dynamics and social interactions'),
  (910, 'Family dynamics at a water-party'),
  (1047, 'Social dynamics at a dinner party'),
  (1707, 'Family life and relationships'),
  (1717, 'dinner arrangement mystery'),
  (1764, 'family bonding and communication'),
  (2187, 'family conflict and reconciliation'),
  (2422, 'social dynamics among women'),
  (2667, "Brother and sister's different behaviors"),
  (2769, 'social dynamics at a party'),
  (2960, 'family dynamics and social status')],
 np.int64(1): [(165, 'young adult fiction books'),
  (464, 'popular young adult literature'),
  (515, "famous children's books")],
 np.int64(2): [(233, "Thomas Edison's inventions and life"),
  (1126, "Ford and Edison's friendship"),
  (1507, "Tesla'

In [ ]:
# def extract_cluster_questions(df, cluster_dialogues):
#     combined_questions = []
    
#     for dialogue_idx, topic in cluster_dialogues:
#         # Get all questions for this dialogue
#         all_questions = df['questions'].loc[dialogue_idx].tolist()
        
#         # Calculate 60% cutoff
#         cutoff_idx = int(len(all_questions) * 0.6)
        
#         # Extract first 60% of questions
#         first_60_percent = all_questions[:cutoff_idx]
        
#         # Add to combined list
#         combined_questions.extend(first_60_percent)
    
#     return combined_questions

# def extract_cluster_last_questions(df, cluster_dialogues):
#     combined_questions = []
    
#     for dialogue_idx, topic in cluster_dialogues:
#         # Get all questions for this dialogue
#         all_questions = df['questions'].loc[dialogue_idx].tolist()
        
#         # Calculate starting index for last 60% (skip first 40%)
#         start_idx = int(len(all_questions) * 0.6)
        
#         # Extract last 60% of questions
#         last_60_percent = all_questions[start_idx:]
        
#         # Add to combined list
#         combined_questions.extend(last_60_percent)
    
#     return combined_questions

In [22]:
def extract_cluster_questions(df, cluster_dialogues):
    cluster_qs = {}
    
    for dialogue_idx, topic in cluster_dialogues:
        # Get all questions for this dialogue
        all_questions = df['questions'].loc[dialogue_idx].tolist()
        
        # Calculate 60% cutoff
        cutoff_idx = int(len(all_questions) * 0.6)
        
        # Extract first 60% of questions
        first_60_percent = all_questions[:cutoff_idx]
        
        # Add to combined list
        cluster_qs[dialogue_idx] = {
            'first_60_percent': first_60_percent
        }
    
    return cluster_qs

def extract_cluster_last_questions(df, cluster_dialogues):
    cluster_qs_test = {}
    
    for dialogue_idx, topic in cluster_dialogues:
        # Get all questions for this dialogue
        all_questions = df['questions'].loc[dialogue_idx].tolist()
        
        # Calculate starting index for last 60% (skip first 40%)
        start_idx = int(len(all_questions) * 0.6)
        
        # Extract last 60% of questions
        last_40_percent = all_questions[start_idx:]
        
        # Add to combined list
        cluster_qs_test[dialogue_idx] = {
            'last_40_percent': last_40_percent
        }
    
    return cluster_qs_test

In [ ]:
# def extract_cluster_aspects(pipe, cluster_topics: list[str], combined_questions: list[str], temperature: float = 0.7) -> list[str]:
#     """Extract common aspects/details of interest from a cluster of similar topics and their combined questions."""
    
#     system_prompt = """
# You are a cluster analysis expert. Your task is to analyze a group of semantically similar topics and their associated questions to identify the key aspects, details, or dimensions that users are consistently interested in across this topic cluster.

# **Instructions:**
# 1. You are given multiple similar topics (from the same cluster) and combined questions from all dialogues in this cluster
# 2. The topics are semantically related and represent the same general domain/subject area
# 3. Analyze the questions to identify recurring themes, specific details, or sub-topics that users frequently ask about
# 4. Focus on aspects that appear across multiple topics/questions in the cluster
# 5. Output ONLY a JSON object with a single "aspects" field containing a list of strings
# 6. Each aspect should be concise (2-6 words) and represent a distinct dimension of interest
# 7. Include only aspects that are clearly supported by the questions and topics

# **Output format (STRICTLY follow this - no other text):**
# {
#   aspects: ["scientific reasoning for existence", "detection methods and technology", "expert predictions and timeline"]
# }

# **Examples:**
# - Topic: "possibility of intelligent life existing on other planets" + Questions about evidence, which planet, methods, timeline, and experts → [age of universe, Earth-like planets,  radio telescopes, timeline for contact]
# - Topic: "university" + Questions about founders, buildings, programs → ["founding history", "campus architecture", "academic programs"]
# """
    
#     topics_formatted = "\n".join([f"- \"{topic}\"" for _, topic in cluster_topics])
#     questions_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(combined_questions)])
    
#     user_prompt = f"""
# **Cluster of Similar Topics:**
# {topics_formatted}

# **Combined Questions from All Dialogues in This Cluster:**
# {questions_formatted}

# Now analyze this topic cluster and extract the key aspects or details that users are consistently interested in across these related topics:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]
    
#     response = pipe(
#         messages,
#         temperature=temperature,
#         do_sample=False,
#         pad_token_id=pipe.tokenizer.eos_token_id
#     )
    
#     # Extract generated text
#     if isinstance(response, list) and len(response) > 0:
#         if isinstance(response[0], dict) and 'generated_text' in response[0]:
#             generated_text = response[0]['generated_text'][-1]['content']
#         else:
#             generated_text = str(response[0])
#     else:
#         generated_text = str(response)
    
#     # Extract JSON from response
#     result = json.loads(generated_text)
#     if "aspects" in result and isinstance(result["aspects"], list):
#         return [str(aspect).strip() for aspect in result["aspects"] if aspect]

In [ ]:
# def generate_cluster_questions(pipe, cluster_topics: list[str], aspects: list[str], asked_questions: list[str], temperature: float = 0.7) -> list[str]:
#     """Generate possible follow-up questions based on a cluster of similar topics, their aspects, and already asked questions."""
    
#     system_prompt = """
# You are a question generation expert specializing in cluster analysis. Your task is to generate new, relevant questions that users might ask across a cluster of semantically similar topics, based on identified aspects of interest and avoiding questions that have already been asked.

# **Instructions:**
# 1. You are given:
#    - Multiple similar topics (from the same semantic cluster)
#    - Key aspects identified as common interests across this topic cluster
#    - Questions that have already been asked across all dialogues in this cluster
# 2. Generate 8 NEW questions that:
#    - Are relevant to the overall domain/theme represented by the cluster
#    - Cover the specified aspects from multiple angles and perspectives
#    - Are fundamentally different from already asked questions (avoid rephrasing the same concepts)
#    - Sound natural and like something real users would ask across different conversations
#    - Explore connections between topics in the cluster or dive deeper into under-explored aspects
# 3. Output ONLY a JSON object with a single "questions" field containing a list of strings
# 4. Each question should be clear, concise, self-contained, and specific

# **Output format (STRICTLY follow this - no other text):**
# {
#   "questions": [
#     "What galaxy do we live in?",
#     "How many stars does it have?",
#     "Does Shostak believe beings from space could make contact with Earth soon?"
#   ]
# }

# **Critical Guidelines:**
# - DO NOT repeat or rephrase any concepts from the already asked questions
# - DO explore aspects from fresh perspectives that haven't been covered yet
# - DO consider how different topics in the cluster might intersect or relate
# - DO make questions specific and actionable rather than vague
# - DO maintain natural, conversational language appropriate for the domain
# """
    
#     topics_formatted = "\n".join([f"- Topic {i+1}: \"{topic}\"" for i, topic in enumerate(cluster_topics)])
#     aspects_formatted = ", ".join([f'"{aspect}"' for aspect in aspects])
#     asked_questions_formatted = "\n".join([f"- \"{q}\"" for q in asked_questions])
    
#     user_prompt = f"""
# **Cluster of Similar Topics:**
# {topics_formatted}

# **Key Aspects of Interest Across This Cluster:**
# {aspects_formatted}

# **Questions Already Asked Across All Dialogues in This Cluster:**
# {asked_questions_formatted}

# Now generate new, relevant questions that users might ask. These questions should explore the cluster domain and aspects in novel ways, avoiding repetition of the already asked questions:
# """
    
#     messages = [
#         {"role": "system", "content": system_prompt.strip()},
#         {"role": "user", "content": user_prompt.strip()}
#     ]

#     try:
#         response = pipe(
#             messages,
#             temperature=temperature,
#             do_sample=True,
#             pad_token_id=pipe.tokenizer.eos_token_id
#         )
        
#         # Extract generated text
#         if isinstance(response, list) and len(response) > 0:
#             if isinstance(response[0], dict) and 'generated_text' in response[0]:
#                 generated_text = response[0]['generated_text'][-1]['content']
#             else:
#                 generated_text = str(response[0])
#         else:
#             generated_text = str(response)
        
#         # Extract JSON from response
#         result = json.loads(generated_text)
#         if "questions" in result and isinstance(result["questions"], list):
#             return [str(q).strip() for q in result["questions"] if q and q.strip()]
#     except json.JSONDecodeError:
#         # Try to find JSON pattern in the response
#         json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
#         if json_match:
#             result = json.loads(json_match.group(0))
#             if "questions" in result and isinstance(result["questions"], list):
#                 return [str(q).strip() for q in result["questions"] if q and q.strip()]

In [ ]:
# def batch_compare_questions(pipe, question_pairs: list[tuple[str, str]], 
#                            batch_size: int = 10, temperature: float = 0.7, 
#                            max_new_tokens: int = 1300) -> dict[tuple[str, str], float]:
#     """
#     Compare multiple question pairs in batches to speed up processing.
    
#     Args:
#         question_pairs: List of (question1, question2) tuples to compare
#         batch_size: Number of pairs to process in each batch
#         temperature: Sampling temperature for generation
#         max_new_tokens: Maximum tokens to generate per batch
    
#     Returns:
#         Dictionary mapping (q1, q2) tuples to confidence scores
#     """
#     results = {}
#     print(len(question_pairs))
    
#     # Process in batches
#     for batch_start in range(0, len(question_pairs), batch_size):
#         batch_end = min(batch_start + batch_size, len(question_pairs))
#         batch_pairs = question_pairs[batch_start:batch_end]
        
#         # Create batch prompt
#         sys_prompt = """
# You are a semantic equivalence evaluator. Your task is to compare multiple pairs of questions and determine the confidence that each pair is semantically equivalent.

# **Instructions:**
# 1. For each question pair, compare the meaning and intent
# 2. Determine if they're asking for the same information and would yield the same answer
# 3. Output ONLY a JSON object with a "results" field containing a list of objects
# 4. Each object must have "question1", "question2", and "confidence" fields
# 5. Confidence must be a number between 0 and 1

# **Output format (STRICTLY follow this - no other text):**
# {
#   "results": [
#     {"question1": "q1 text", "question2": "q2 text", "confidence": 0.85},
#     {"question1": "q1 text", "question2": "q2 text", "confidence": 0.32}
#   ]
# }

# **Explanation of confidence scores:**
# - 0.9-1.0: Definitely equivalent (same meaning, intent, and expected answer)
# - 0.7-0.9: Highly likely equivalent (minor wording differences only)
# - 0.5-0.7: Possibly equivalent (similar intent but different phrasing)
# - 0.3-0.5: Unlikely equivalent (related but different aspects)
# - 0.0-0.3: Definitely different (completely different questions)
# """
        
#         # Format batch pairs
#         pairs_formatted = []
#         for i, (q1, q2) in enumerate(batch_pairs):
#             pairs_formatted.append(f"Pair {i+1}:\nQuestion 1: \"{q1}\"\nQuestion 2: \"{q2}\"\n")
        
#         user_prompt = f"""
# **Question pairs to compare:**
# {chr(10).join(pairs_formatted)}

# Now evaluate all these question pairs and provide confidence scores for each:
# """
        
#         messages = [
#             {"role": "system", "content": sys_prompt.strip()},
#             {"role": "user", "content": user_prompt.strip()}
#         ]
        
#         try:
#             # Single LLM call for the entire batch
#             response = pipe(
#                 messages,
#                 temperature=temperature,
#                 max_new_tokens = max_new_tokens,
#                 do_sample=False,
#                 pad_token_id=pipe.tokenizer.eos_token_id
#             )
            
#             # Extract generated text
#             generated_text = response[0]['generated_text'][-1]['content'] if isinstance(response, list) else str(response)
            
#             # Parse JSON response
#             try:
#                 batch_results = json.loads(generated_text)
#                 if "results" in batch_results and isinstance(batch_results["results"], list):
#                     for item in batch_results["results"]:
#                         if all(key in item for key in ["question1", "question2", "confidence"]):
#                             q1 = item["question1"].strip()
#                             q2 = item["question2"].strip()
#                             confidence = float(item["confidence"])
#                             results[(q1, q2)] = max(0.0, min(1.0, confidence))
#             except (json.JSONDecodeError, ValueError) as e:
#                 print(f"Error parsing batch results: {e}")
#                 print(f"Raw response: {generated_text}")
#         except Exception as e:
#             print(f"Error in batch comparison: {e}")

#     return results

# #         try:
# #             response = pipe(
# #                 messages,
# #                 temperature=temperature,
# #                 max_new_tokens=max_new_tokens,
# #                 do_sample=False,
# #                 pad_token_id=pipe.tokenizer.eos_token_id
# #             )
            
# #             generated_text = response[0]['generated_text'][-1]['content'] if isinstance(response, list) else str(response)
            
# #             # Try to parse JSON with multiple fallback strategies
# #             batch_results = parse_json_with_fallbacks(generated_text, batch_pairs)
            
# #             if batch_results and "results" in batch_results and isinstance(batch_results["results"], list):
# #                 for item in batch_results["results"]:
# #                     try:
# #                         if all(key in item for key in ["question1", "question2", "confidence"]):
# #                             q1 = str(item["question1"]).strip()
# #                             q2 = str(item["question2"]).strip()
# #                             confidence = float(item["confidence"])
# #                             results[(q1, q2)] = max(0.0, min(1.0, confidence))
# #                     except (ValueError, TypeError) as e:
# #                         print(f"Error processing individual result: {e}")
# #                         print(f"Problematic item: {item}")
# #             else:
# #                 print(f"Invalid batch results structure: {batch_results}")
                
# #         except Exception as e:
# #             print(f"Error in batch comparison: {e}")
# #             print(f"Raw response that caused error: {generated_text}")
    
# #     return results

# # def parse_json_with_fallbacks(json_text: str, expected_pairs: list) -> dict:
# #     """
# #     Attempt to parse JSON with multiple fallback strategies for common LLM errors.
# #     """
# #     # Strategy 1: Try direct parsing
# #     try:
# #         return json.loads(json_text)
# #     except json.JSONDecodeError as e1:
# #         print(f"Direct JSON parse failed: {e1}")
    
# #     # Strategy 2: Extract JSON-like content using regex
# #     try:
# #         # Look for content between { and }
# #         json_match = re.search(r'\{[\s\S]*\}', json_text)
# #         if json_match:
# #             extracted_json = json_match.group(0)
# #             try:
# #                 return json.loads(extracted_json)
# #             except json.JSONDecodeError as e2:
# #                 print(f"Regex extraction parse failed: {e2}")
# #     except Exception as e2:
# #         print(f"Regex extraction failed: {e2}")
    
# #     # Strategy 3: Fix common JSON errors manually
# #     try:
# #         # Fix the specific error seen in the example: extra } before confidence
# #         fixed_text = re.sub(r'"}\s*,\s*"confidence"', r'", "confidence"', json_text)
# #         # Fix missing quotes around confidence values
# #         fixed_text = re.sub(r':\s*(\d+\.\d+)}', r': "\1"}', fixed_text)
# #         # Fix trailing commas
# #         fixed_text = re.sub(r',\s*}', '}', fixed_text)
# #         fixed_text = re.sub(r',\s*\]', ']', fixed_text)
        
# #         try:
# #             return json.loads(fixed_text)
# #         except json.JSONDecodeError as e3:
# #             print(f"Manual fixing parse failed: {e3}")
# #     except Exception as e3:
# #         print(f"Manual fixing failed: {e3}")
    
# #     # Strategy 4: Last resort - try to extract confidence values manually
# #     try:
# #         results = []
# #         for i, (q1, q2) in enumerate(expected_pairs):
# #             # Look for confidence values in the text
# #             pattern = f'"{q1}"[^{{}}]*"{q2}"[^{{}}]*"confidence":\s*([\d\.]+)'
# #             match = re.search(pattern, json_text, re.IGNORECASE | re.DOTALL)
# #             if match:
# #                 confidence = float(match.group(1))
# #                 results.append({
# #                     "question1": q1,
# #                     "question2": q2,
# #                     "confidence": confidence
# #                 })
        
# #         if results:
# #             return {"results": results}
# #     except Exception as e4:
# #         print(f"Manual extraction failed: {e4}")
    
# #     print("All JSON parsing strategies failed. Returning empty results.")
# #     return {"results": []}

In [12]:
def extract_aspects_cluster(pipe, current_topic: str, current_questions: list[str], cluster_questions: list[list[str]], temperature: float = 0.7) -> list[str]:
    """Extract detailed aspects of a topic using context from all dialogues in the same cluster."""
    
    system_prompt = """
You are an aspect extraction expert specializing in contextual analysis. Your task is to identify specific details and dimensions of a topic by analyzing questions from:
1. The current dialogue (primary focus)
2. Related dialogues in the same topic cluster (providing broader context)

**Instructions:**
1. Analyze ALL provided questions together - they all relate to the same core topic cluster
2. Identify specific aspects, details, or dimensions of the main topic that users are interested in
3. Focus on concrete details mentioned or implied across the dialogues
4. Prioritize aspects that appear in multiple dialogues OR are deeply explored in the current dialogue
5. Output ONLY a JSON object with a single "aspects" field containing a list of strings
6. Each aspect should be concise (2-6 words) and represent a distinct detail or sub-topic
7. Include only aspects strongly supported by the questions

**Output format (STRICTLY follow this - no other text):**
{
  "aspects": ["album release dates", "auction bidding processes", "memorabilia authentication"]
}

**Critical Guidelines:**
- LEVERAGE CLUSTER CONTEXT: Use questions from related dialogues to discover aspects not explicitly mentioned in the current dialogue
- AVOID GENERIC ASPECTS: Be specific and concrete (e.g., "vinyl condition grading" not just "records")
- PRIORITIZE RECURRING THEMES: Aspects appearing across multiple dialogues are more significant
- MAINTAIN TOPIC FOCUS: All aspects must directly relate to the core topic cluster
"""
    
    # Format current dialogue questions
    current_formatted = "\n".join([f"- Question {i+1}: \"{q}\"" for i, q in enumerate(current_questions)])  # Limit to 10 for token efficiency
    
    # Format cluster context questions (from other dialogues)
    cluster_context = []
    for i, dialogue_questions in enumerate(cluster_questions[:3]):  # Limit to 3 dialogues
        dialogue_sample = dialogue_questions  # Limit to 5 questions per dialogue
        cluster_context.append(f"Related Dialogue {i+1} questions:\n" + "\n".join([f"  • \"{q}\"" for q in dialogue_sample]))
    cluster_formatted = "\n\n".join(cluster_context)
    
    user_prompt = f"""
**CORE TOPIC CLUSTER:** "{current_topic}"

**CURRENT DIALOGUE QUESTIONS (primary focus):**
{current_formatted}

**ADDITIONAL CONTEXT FROM RELATED DIALOGUES (same topic cluster):**
{cluster_formatted}

Analyze ALL questions above to extract the most significant specific aspects of "{current_topic}" that users care about:
"""
    
    messages = [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

    response = pipe(
        messages,
        temperature=temperature,
        do_sample=False,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    
    # Extract generated text
    if isinstance(response, list) and len(response) > 0:
        if isinstance(response[0], dict) and 'generated_text' in response[0]:
            generated_text = response[0]['generated_text'][-1]['content']
        else:
            generated_text = str(response[0])
    else:
        generated_text = str(response)
    
    # Extract JSON from response
    result = json.loads(generated_text)
    if "aspects" in result and isinstance(result["aspects"], list):
        return [str(aspect).strip() for aspect in result["aspects"] if aspect]

In [28]:
# cluster_questions_dict = extract_cluster_questions(df, clusters[1])
# clusters[1]

[(165, 'young adult fiction books'),
 (464, 'popular young adult literature'),
 (515, "famous children's books")]

In [29]:
# dialogue_aspects = {}
# for dialogue_idx, topic in clusters[1]:
#         # Get current dialogue questions
#         current_questions = cluster_questions_dict[dialogue_idx]['first_60_percent']
        
#         # Prepare cluster context: questions from OTHER dialogues in cluster
#         cluster_context_questions = [
#             cluster_questions_dict[other_idx]['first_60_percent'] 
#             for other_idx in cluster_questions_dict.keys() 
#             if other_idx != dialogue_idx
#         ]

#         dialogue_aspects[dialogue_idx] = extract_aspects_cluster(
#             pipe,
#             current_topic=topic,
#             current_questions=current_questions,
#             cluster_questions=cluster_context_questions
#         )
# cluster_results = {"dialogues": {}}

# for dialogue_idx, topic in clusters[1]:
#     # Get current dialogue data
#     current_questions = cluster_questions_dict[dialogue_idx]['first_60_percent']
#     current_aspects = dialogue_aspects[dialogue_idx]
    
#     # Prepare cluster context for generation: aspects + questions from OTHER dialogues
#     cluster_context = []
#     for other_idx, other_topic in clusters[1]:
#         if other_idx != dialogue_idx:
#             cluster_context.append({
#                 'aspects': dialogue_aspects[other_idx],
#                 'questions': cluster_questions_dict[other_idx]['first_60_percent']
#             })
    
#     # Generate questions using enriched context
#     gen_questions = generate_questions_cluster(
#         pipe,
#         current_topic=topic,
#         current_aspects=current_aspects,
#         current_questions=current_questions,
#         cluster_context=cluster_context
#     )
#     print(f"  Generated {len(gen_questions)} questions for dialogue {dialogue_idx}")
    
#     # Get remaining questions (last 40%) for THIS dialogue only
#     remaining_questions = extract_cluster_last_questions(df, [(dialogue_idx, topic)])
#     dialogue_remaining = remaining_questions[dialogue_idx]['last_40_percent']
    
#     # Compare generated questions with remaining questions
#     comparisons = {}
#     for q_a in gen_questions:
#         comparisons[q_a] = {}
#         for q_b in dialogue_remaining:
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence
    
#     # Store results for this dialogue
#     cluster_results["dialogues"][dialogue_idx] = {
#         "topic": topic,
#         "aspects": current_aspects,
#         "gen_questions": gen_questions,
#         "comparisons": comparisons
#     }


  Generated 8 questions for dialogue 165
  Generated 8 questions for dialogue 464
  Generated 8 questions for dialogue 515


In [27]:
def generate_questions_cluster(pipe, current_topic: str, current_aspects: list[str], current_questions: list[str], cluster_context: list[dict], temperature: float = 0.7, max_new_tokens: int = 500) -> list[str]:
    """Generate context-aware questions for a dialogue using enriched cluster information."""
    
    system_prompt = """
You are an expert question generator specializing in contextual dialogue expansion. Your task is to generate new, relevant questions for a user by leveraging:
1. The current dialogue's specific focus (primary priority)
2. Broader context from related dialogues in the same topic cluster

**Instructions:**
1. Analyze ALL provided information about the topic cluster
2. Generate 8 NEW questions that:
   - Are highly relevant to the CURRENT dialogue's specific aspects and questions
   - Incorporate valuable details discovered from OTHER dialogues in the cluster
   - Avoid repeating or rephrasing questions already asked in the current dialogue
   - Explore unexplored dimensions of the topic revealed by the cluster context
   - Sound natural and conversational (like a real user would ask)
3. Prioritize questions that:
   - Bridge the current dialogue's focus with complementary aspects from cluster context
   - Address obvious gaps where cluster context reveals important aspects missing in current dialogue
   - Maintain specificity to the current dialogue's unique angle within the broader topic
4. Output ONLY a JSON object with a single "questions" field containing a list of strings
5. Each question must be clear, concise, and self-contained

**Output format (STRICTLY follow this - no other text):**
{
  "questions": [
    "What authentication methods are used for rare Michael Jackson memorabilia?",
    "How do posthumous album releases affect the value of original memorabilia?",
    "What was the highest bid ever recorded for Jackson's signature glove?"
  ]
}

**Critical Guidelines:**
- CURRENT DIALOGUE IS PRIMARY: Never generate questions irrelevant to the current dialogue's specific focus
- CLUSTER CONTEXT IS ENRICHMENT: Use other dialogues to discover NEW angles, not to change the core topic
- AVOID REPETITION: Do not generate questions semantically equivalent to already asked questions
- BE SPECIFIC: Leverage concrete details from cluster aspects
- MAINTAIN CONVERSATIONAL FLOW: Questions should feel like natural follow-ups to the current discussion
"""
    
    # Format current dialogue information
    current_aspects_formatted = ", ".join([f'"{a}"' for a in current_aspects])
    current_questions_formatted = "\n".join([f"- \"{q}\"" for q in current_questions])
    
    # Format cluster context (only include relevant parts from other dialogues)
    cluster_context_formatted = []
    for i, context in enumerate(cluster_context[:3]):  # Limit to 2 other dialogues for token efficiency
        aspects = context.get('aspects', [])
        questions = context.get('questions', [])
        
        if aspects or questions:
            dialogue_context = f"Related Dialogue {i+1}:\n"
            if aspects:
                dialogue_context += f"Key aspects: {', '.join([f'\"{a}\"' for a in aspects])}\n"
            if questions:
                dialogue_context += "Sample questions:\n" + "\n".join([f"  • \"{q}\"" for q in questions])
            cluster_context_formatted.append(dialogue_context)
    
    cluster_formatted = "\n\n".join(cluster_context_formatted)
    
    user_prompt = f"""
**CURRENT DIALOGUE FOCUS**
Topic: "{current_topic}"
Specific Aspects: {current_aspects_formatted}
Already Asked Questions:
{current_questions_formatted}

**ENRICHMENT FROM TOPIC CLUSTER CONTEXT**
{cluster_formatted}

Generate 8 NEW questions that would naturally follow in the CURRENT dialogue, leveraging insights from the cluster context while staying focused on the current dialogue's specific aspects:
"""
    
    messages = [
        {"role": "system", "content": system_prompt.strip()},
        {"role": "user", "content": user_prompt.strip()}
    ]

    response = pipe(
        messages,
        temperature=temperature,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    
    # Extract generated text
    if isinstance(response, list) and len(response) > 0:
        if isinstance(response[0], dict) and 'generated_text' in response[0]:
            generated_text = response[0]['generated_text'][-1]['content']
        else:
            generated_text = str(response[0])
    else:
        generated_text = str(response)
    
    # Extract JSON from response
    try:
        result = json.loads(generated_text)
        if "questions" in result and isinstance(result["questions"], list):
            # Filter out empty questions and limit to 20
            questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
            return questions
    except json.JSONDecodeError:
        # Try to find JSON pattern in the response
        json_match = re.search(r'\{[^{}]*"questions"[^{}]*\}', generated_text)
        if json_match:
            result = json.loads(json_match.group(0))
            if "questions" in result and isinstance(result["questions"], list):
                questions = [str(q).strip() for q in result["questions"] if q and q.strip()]
                return questions

In [ ]:
results_app2={}

In [ ]:
# for cl in range(len(clusters)):
#     print(cl)
#     cluster_questions = extract_cluster_questions(df, clusters[cl])

#     cluster_aspects = extract_cluster_aspects(pipe, clusters[cl], cluster_questions)
    
#     gen_questions = generate_cluster_questions(pipe, clusters[cl], cluster_aspects, cluster_questions)

#     comparisons = {}
#     remaining_questions = extract_cluster_last_questions(df, clusters[cl])
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     results_app2[f'cluster {cl}'] = {
#     "topic": clusters[cl],
#     "aspects": cluster_aspects,
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
# for cl in range(len(clusters)):
#     print(cl)
#     cluster_questions = extract_cluster_questions(df, clusters[cl])

#     cluster_aspects = extract_cluster_aspects(pipe, clusters[cl], cluster_questions)
    
#     gen_questions = generate_cluster_questions(pipe, clusters[cl], cluster_aspects, cluster_questions)

#     comparisons = {}
#     remaining_questions = extract_cluster_last_questions(df, clusters[cl])
#     for i, q_a in enumerate(gen_questions):
#         comparisons[q_a] = {}
#         for j, q_b in enumerate(remaining_questions):
#             confidence = compare_questions(pipe, q_a, q_b)
#             comparisons[q_a][q_b] = confidence

#     results_app2[f'cluster {cl}'] = {
#     "topic": clusters[cl],
#     "aspects": cluster_aspects,
#     "gen_questions": gen_questions,
#     "comparisons": comparisons
#     }

In [ ]:
for cl_idx, cluster_dialogues in enumerate(clusters.values()):
    print(f"Processing cluster {cl_idx} with {len(cluster_dialogues)} dialogues")
    
    # Get first 60% questions for all dialogues in this cluster
    cluster_questions_dict = extract_cluster_questions(df, cluster_dialogues)
    
    # Pre-extract aspects for each dialogue using cluster context
    dialogue_aspects = {}
    for dialogue_idx, topic in cluster_dialogues:
        # Get current dialogue questions
        current_questions = cluster_questions_dict[dialogue_idx]['first_60_percent']
        
        # Prepare cluster context: questions from OTHER dialogues in cluster
        cluster_context_questions = [
            cluster_questions_dict[other_idx]['first_60_percent'] 
            for other_idx in cluster_questions_dict.keys() 
            if other_idx != dialogue_idx
        ]
        
        # Extract aspects using enriched cluster context
        dialogue_aspects[dialogue_idx] = extract_aspects_cluster(
            pipe,
            current_topic=topic,
            current_questions=current_questions,
            cluster_questions=cluster_context_questions
        )
    
    # Generate questions and perform comparisons for each dialogue
    cluster_results = {"dialogues": {}}

    for dialogue_idx, topic in cluster_dialogues:
            
            # Get current dialogue data
            current_questions = cluster_questions_dict[dialogue_idx]['first_60_percent']
            current_aspects = dialogue_aspects[dialogue_idx]
            
            # Prepare cluster context for generation: aspects + questions from OTHER dialogues
            cluster_context = []
            for other_idx, other_topic in cluster_dialogues:
                if other_idx != dialogue_idx:
                    cluster_context.append({
                        'aspects': dialogue_aspects[other_idx],
                        'questions': cluster_questions_dict[other_idx]['first_60_percent']
                    })
            
            # Generate questions using enriched context
            gen_questions = generate_questions_cluster(
                pipe,
                current_topic=topic,
                current_aspects=current_aspects,
                current_questions=current_questions,
                cluster_context=cluster_context
            )
            
            # Get remaining questions (last 40%) for THIS dialogue only
            remaining_questions = extract_cluster_last_questions(df, [(dialogue_idx, topic)])
            dialogue_remaining = remaining_questions[dialogue_idx]['last_40_percent']
            
            # Compare generated questions with remaining questions
            comparisons = {}
            for q_a in gen_questions:
                comparisons[q_a] = {}
                for q_b in dialogue_remaining:
                    confidence = compare_questions(pipe, q_a, q_b)
                    comparisons[q_a][q_b] = confidence
            
            # Store results for this dialogue
            cluster_results["dialogues"][dialogue_idx] = {
                "topic": topic,
                "aspects": current_aspects,
                "gen_questions": gen_questions,
                "comparisons": comparisons
            }
        
    # Store cluster results
    results_app2[f'cluster {cl_idx}'] = cluster_results

In [ ]:
# clusters_number = [11, 12, 14, 15, 16, 18, 19]

In [ ]:
# for cl in clusters_number:
#     print(cl)
#     cluster_questions = extract_cluster_questions(df, clusters[cl])
#     cluster_aspects = extract_cluster_aspects(pipe, clusters[cl], cluster_questions)
#     gen_questions = generate_cluster_questions(pipe, clusters[cl], cluster_aspects, cluster_questions)
    
#     # Create all question pairs for comparison
#     remaining_questions = extract_cluster_last_questions(df, clusters[cl])
#     question_pairs = [(q_a, q_b) for q_a in gen_questions for q_b in remaining_questions]
    
#     # Batch comparison - much faster!
#     batch_results = batch_compare_questions(pipe, question_pairs, batch_size=10)
    
#     # Reconstruct comparisons dictionary from batch results
#     comparisons = {q_a: {} for q_a in gen_questions}
#     for (q_a, q_b), confidence in batch_results.items():
#         comparisons[q_a][q_b] = confidence
    
#     results_app2[f'cluster {cl}'] = {
#         "topic": clusters[cl],
#         "aspects": cluster_aspects,
#         "gen_questions": gen_questions,
#         "comparisons": comparisons
#     }

In [ ]:
# check1 = ['What kind of materials are stored in the secret archives beyond printed books?',
#           'Are there any restrictions on who can access the secret archives?',
#           'What historical events influenced the decision to move the secret archives?',
#           'How has the purpose of the Vat evolved since its opening?',
#           'Are there any known classified topics or forbidden subjects in the secret collection?',
#           'Has the size of the secret archives grown over time?']

# check2 = ['What is the Vat the library of?',
#           'How many books survived the Pre Lateran period?',
#           'how many printed books does it contain?',
#           'when were the Secret Archives moved from the rest of the library?',
#           'what is the point of the project started in 2014?',
#           'what will this allow?']
# row=1
# question_pairs = [(q_a, q_b) for q_a in check1 for q_b in check2]

# batch_results = batch_compare_questions(pipe, question_pairs, batch_size=10)

# comparisons = {q_a: {} for q_a in check1}
# for (q_a, q_b), confidence in batch_results.items():
#     comparisons[q_a][q_b] = confidence

# results_app2[f'row {row}'] = {
#     "topic": clusters[0],
#     "gen_questions": check1,
#     "comparisons": comparisons
# }
    

In [ ]:
results_app2_df = pd.DataFrame(results_app2)

In [ ]:
results_app2_df.T.to_csv('/kaggle/working/results_app2_df.csv')

In [ ]:
# results_df.T['comparisons'].iloc[4]

In [ ]:
# next(iter(next(iter(results_df.T['comparisons'].iloc[0].values())).items()))

In [ ]:
# messages = [
#     {"role": "system", "content": sys_prompt.strip()},
#     {"role": "user", "content": prompt.strip()}
# ]
# pipe(messages)